In [2]:
# Dependencies
import pandas as pd
import requests
import json
import os

# Google API Key (maybe later)
from config import gkey


In [3]:
# Source files
attendance_raw = 'Resources/national_park_attendance_Prev3Y.xlsx'
park_info = 'Resources/national_park_locations.xlsx'
#wildfires = 'Resources/California_Fire_Incidents.csv'

# Read data and store in dataframe
attendance_data = pd.read_excel(attendance_raw)
park_info_data = pd.read_excel(park_info)
#wildfire_data = pd.read_csv(wildfires)


In [4]:
# Just looking at wildfire data
#wildfire_data.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [38]:
# Limit dataframe to California & Florifa
national_parks = park_info_data.loc[(park_info_data['ParkType']=='National Park')]
focus_parks = pd.DataFrame(national_parks.loc[(park_info_data['State'] == 'CA') | (park_info_data['State'] == 'FL')])
focus_parks.reset_index()

#focus_parks['FullName'] = focus_parks['Park']

# Create new 'FullName' column (empty)
focus_parks['FullName'] = focus_parks['Park'].str.replace(['NP'],'')#+focus_parks['ParkType'])

focus_parks['FullName']=focus_parks['Park'].str[:-2]+focus_parks['ParkType']

#focus_parks['Park'].replace('NP', 'National Park', inplace=True)
#focus_parks['Park'] = focus_parks['Park'].str.replace('NP','')

focus_parks.head(20)

,Park,UnitCode,ParkType,Region,State,FullName
30,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park
63,Channel Islands NP,CHIS,National Park,Pacific West,CA,Channel Islands National Park
87,Death Valley NP,DEVA,National Park,Pacific West,CA,Death Valley National Park
93,Dry Tortugas NP,DRTO,National Park,Southeast,FL,Dry Tortugas National Park
101,Everglades NP,EVER,National Park,Southeast,FL,Everglades National Park
191,Joshua Tree NP,JOTR,National Park,Pacific West,CA,Joshua Tree National Park
198,Kings Canyon NP,KICA,National Park,Pacific West,CA,Kings Canyon National Park
210,Lassen Volcanic NP,LAVO,National Park,Pacific West,CA,Lassen Volcanic National Park
281,Pinnacles NP,PINN,National Park,Pacific West,CA,Pinnacles National Park
293,Redwood NP,REDW,National Park,Pacific West,CA,Redwood National Park


In [42]:
# Combine the data into a single dataframe  
park_data_merge = pd.merge(focus_parks, attendance_data, how="left", on=["Park", "Park"])
park_data_merge.head(20)

,Park,UnitCode,ParkType,Region,State,FullName,Code,Month,Visitors
0,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,BISC,2020-01-01 00:00:00,79041.0
1,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,BISC,2020-02-01 00:00:00,91655.0
2,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,BISC,2020-03-01 00:00:00,10962.0
3,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,BISC,2020-04-01 00:00:00,2358.0
4,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,BISC,2020-05-01 00:00:00,43692.0
5,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,BISC,2020-06-01 00:00:00,32646.0
6,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,BISC,2020-07-01 00:00:00,28770.0
7,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,BISC,2020-08-01 00:00:00,34416.0
8,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,BISC,2020-09-01 00:00:00,18276.0
9,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,BISC,2020-10-01 00:00:00,17436.0


In [7]:
# Find the geocoordinates (latitude and longitude)
park = "Yellowstone"

params = {"address": park, "key": gkey}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

#print("Drill #1: The Geocoordinates of X")

# Run request
response = requests.get(base_url, params=params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

# Convert to JSON
park_geo = response.json()

# Extract lat/lng
lat = park_geo["results"][0]["geometry"]["location"]["lat"]
lng = park_geo["results"][0]["geometry"]["location"]["lng"]

# Print results
print(f"{park}: {lat}, {lng}")

Drill #1: The Geocoordinates of X
Yellowstone: 44.427963, -110.588455


In [45]:
# Build URL using the Google Maps API
lats=[]
longs=[]

base_url = "https://maps.googleapis.com/maps/api/geocode/json"

for Park in focus_parks['FullName']:
    params = {"address": Park, "key": gkey}
    response = requests.get(base_url, params=params)
    park_geo = response.json()
    lats.append(park_geo["results"][0]["geometry"]["location"]["lat"])
    longs.append(park_geo["results"][0]["geometry"]["location"]["lng"])
 
                
    

In [48]:
# Add geocoordinates
focus_parks['Latitude']=lats
focus_parks['Longitude']=longs
focus_parks.head()


,Park,UnitCode,ParkType,Region,State,FullName,Latitude,Longitude
30,Biscayne NP,BISC,National Park,Southeast,FL,Biscayne National Park,25.482423,-80.208310
63,Channel Islands NP,CHIS,National Park,Pacific West,CA,Channel Islands National Park,34.006936,-119.778533
87,Death Valley NP,DEVA,National Park,Pacific West,CA,Death Valley National Park,36.505389,-117.079408
93,Dry Tortugas NP,DRTO,National Park,Southeast,FL,Dry Tortugas National Park,24.633333,-82.920000
101,Everglades NP,EVER,National Park,Southeast,FL,Everglades National Park,25.745929,-80.554956
